In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from multiprocessing import cpu_count
import tensorflow as tf

Using TensorFlow backend.


In [2]:
nb_classes = 20
nb_cpus = cpu_count()//2
nb_gpus = 4

image_size = (299, 299)
input_shape= (299, 299, 3)

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1.0)(p_out)
    predictions = Dense(nb_classes, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    
model = multi_gpu_model(model, gpus=nb_gpus)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_5 (

In [4]:
model.load_weights('weights_001_0.2472.hdf5')

In [5]:
batch_size = 128
test_path = "/home/cnn/Documents/batch6.1/train6/valid"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_path, 
                                         target_size=image_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
all_test_results = model.predict_generator(test_generator, 
                                           len(test_generator), 
                                           workers=nb_cpus, 
                                           use_multiprocessing=True,
                                           verbose=1)
all_labels = test_generator.classes

Found 38043 images belonging to 20 classes.
298/298 [==============================] - 150s 505ms/step


In [6]:
class_label_dict = test_generator.class_indices
print(test_generator.class_indices)
def get_key(dict_, value):
    return [k for k, v in dict_.items() if v == value]

# create class num lens dict, every dict store current class predict num
total_predictions_dict = {}
for row_class_name, label in class_label_dict.items():
    total_predictions_dict[row_class_name] = {}
    for column_class_name, label in class_label_dict.items():
        total_predictions_dict[row_class_name][column_class_name] = 0

{'TRI': 18, 'RC': 14, 'GEC': 7, 'SC': 15, 'ACTINO': 0, 'HSIL_S': 10, 'AGC_A': 1, 'SCC_R': 17, 'HSIL_M': 9, 'SCC_G': 16, 'VIRUS': 19, 'CC': 4, 'ASCUS': 3, 'MC': 13, 'EC': 5, 'HSIL_B': 8, 'AGC_B': 2, 'LSIL_E': 11, 'LSIL_F': 12, 'FUNGI': 6}


In [16]:
thresh = 0.95

for i, label in enumerate(all_labels):
    predict_index = np.argmax(all_test_results[i])
    predict_det = all_test_results[i][predict_index]
    if (predict_det > thresh):
        # get the first result
        label_class_name = get_key(class_label_dict, label)[0]
        test_class_name = get_key(class_label_dict, np.argmax(all_test_results[i]))[0]
#     print(label_class_name)
#     print(total_predictions_dict[label_class_name].keys())
        total_predictions_dict[label_class_name][test_class_name] += 1

In [17]:
print(total_predictions_dict['RC'])

{'TRI': 0, 'SCC_R': 0, 'GEC': 9, 'SC': 0, 'ACTINO': 0, 'HSIL_S': 0, 'AGC_A': 0, 'CC': 0, 'EC': 3, 'SCC_G': 0, 'VIRUS': 0, 'RC': 436, 'ASCUS': 0, 'MC': 0, 'HSIL_M': 0, 'HSIL_B': 0, 'AGC_B': 0, 'LSIL_E': 0, 'LSIL_F': 0, 'FUNGI': 0}


In [18]:
import csv
out = open('confusion_matrix.csv','a', newline='')
csv_write = csv.writer(out,dialect='excel')

# write the title
line = [class_name for class_name, label in class_label_dict.items()]
line = [" "] + line + ["TOTAL"] + ["ACC"]
csv_write.writerow(line)

# write rows

true_num = 0
all_num = 0

for row_class_name, label in class_label_dict.items():
    one_class_total_predict = 0
    line = [row_class_name]
    for column_class_name, label in class_label_dict.items():
        one_class_total_predict += total_predictions_dict[row_class_name][column_class_name]
    
    for column_class_name, label in class_label_dict.items():
        one_class_cur_predict = total_predictions_dict[row_class_name][column_class_name]
        # acc
        #acc = round((one_class_cur_predict / one_class_total_predict), 4)
        #line.append(str(acc))
        # num
        line.append(one_class_cur_predict)
    print(one_class_total_predict)
    print(str(one_class_total_predict))
    line.append(str(one_class_total_predict))
    line.append(round((total_predictions_dict[row_class_name][row_class_name] / one_class_total_predict), 4))       
    print(line)
    csv_write.writerow(line)
    
    true_num += total_predictions_dict[row_class_name][row_class_name]
    all_num += one_class_total_predict
    
csv_write.writerow(["ALL_ACC"] + [round((true_num / all_num), 4)])
    
out.close()

33658
33658
['TRI', 33646, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, '33658', 0.9996]
448
448
['RC', 0, 436, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, '448', 0.9732]
5913
5913
['GEC', 0, 18, 5732, 2, 3, 17, 19, 0, 17, 11, 2, 0, 4, 0, 15, 31, 40, 0, 2, 0, '5913', 0.9694]
4132
4132
['SC', 25, 0, 1, 3358, 0, 716, 0, 0, 1, 2, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, '4132', 0.8127]
7189
7189
['ACTINO', 0, 0, 0, 0, 7130, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '7189', 0.9918]
18022
18022
['HSIL_S', 1, 0, 4, 320, 0, 17521, 4, 16, 13, 113, 23, 0, 7, 0, 0, 0, 0, 0, 0, 0, '18022', 0.9722]
2949
2949
['AGC_A', 0, 0, 27, 0, 0, 0, 2858, 0, 7, 27, 0, 0, 0, 0, 0, 20, 6, 0, 4, 0, '2949', 0.9691]
5407
5407
['SCC_R', 0, 0, 0, 1, 0, 42, 0, 5360, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, '5407', 0.9913]
5984
5984
['HSIL_M', 0, 0, 6, 0, 0, 26, 20, 0, 5863, 11, 0, 0, 0, 0, 0, 34, 23, 0, 1, 0, '5984', 0.9798]
3214
3214
['SCC_G', 0, 0, 24, 8, 0, 232, 5, 23, 4, 2890, 0, 0, 15, 1, 0, 0, 0, 2, 